In [1]:
import os
import pandas as pd
import numpy as np

In [28]:
# reading the dataset
PImPo = pd.read_csv("PImPo.csv",delimiter=";")
PImPo.rename(columns={"Unnamed: 0":"id",1:"country",2:"party",3:"content",4:"target",5:"stance"},inplace=True)
PImPo.drop(columns=["id"],inplace=True)
PImPo

,country,party,content,target,stance
0,sweden,swe: Green Ecology Party,Vi vill ge människor möjligheten att leva i et...,Integration,FAVOR
1,sweden,swe: Green Ecology Party,och där lika rättigheter för alla är en självk...,Integration,FAVOR
2,sweden,swe: Green Ecology Party,Flyktingpolitiken ska göras human,Immigration,FAVOR
3,sweden,swe: Green Ecology Party,Förslagen i vårt valmanifest grundar sig i en ...,Immigration,FAVOR
4,sweden,swe: Green Ecology Party,"Alla barn har rätt till ett liv i kärlek, resp...",Integration,FAVOR
...,...,...,...,...,...
8955,new zealand,nzl: Maori Party,The Office of the Controller and Auditor Gener...,Integration,NONE
8956,new zealand,nzl: Maori Party,"In return, we expect services to meet the need...",Integration,FAVOR
8957,new zealand,nzl: Maori Party,All new citizens to complete a course in the h...,Integration,NONE
8958,new zealand,nzl: Mana Party,"Maori, Pasifika and migrant peoples are dispro...",Integration,FAVOR


In [47]:
# use for additional filters
country = "norway"
PImPo = PImPo.loc[PImPo["country"]==country].drop(columns=["country"])

if not os.path.exists("./"+country):
    os.mkdir("./"+country)

In [29]:
# no country
country = "all"
PImPo = PImPo.drop(columns=["country"])

In [30]:
# use for test/train split
from sklearn.model_selection import train_test_split
PImPo_train, PImPo_test = train_test_split(PImPo,test_size=0.1)

In [31]:
PImPo_train.to_csv(country+"/train.csv",index=False, sep=",")
PImPo_test.to_csv(country+"/test.csv",index=False, sep=",")



In [32]:
PImPo_train

,party,content,target,stance
3231,nld: Reformed Political Party,Gebeuren er op islamitische scholen of in mosk...,Integration,AGAINST
2695,nld: Green Left,"Jongeren, vrouwen en migranten krijgen meer ka...",Integration,FAVOR
198,swe: Social Democratic Labour Party,"Alla arbetstagare ska behandlas lika, oavsett ...",Integration,FAVOR
3742,nld: Christian Union,De uitvoering van de pardonregeling heeft gele...,Immigration,NONE
6407,deu: Alliance‘90/Greens,Da die Konflikte in Afghanistan nicht militäri...,Immigration,FAVOR
...,...,...,...,...
1094,nor: Socialist Left Party,SV vil styrke arbeidet for å sikre alle barn e...,Integration,FAVOR
6794,deu: Christian Democratic Union/C.Social Union,Leider gibt es eine wachsende Zahl von Kindern...,Integration,NONE
6739,deu: Free Democratic Party,- Deutschland hat die Bemühungen um die Verb...,Immigration,FAVOR
6301,deu: Alliance‘90/Greens,Wir wollen die Abschiebehaft abschaffen.,Immigration,AGAINST
